In [1]:
import numpy as np
import cv2

# Stage 1

Stage 1 of this task is concerned with blending an object, taken from one image, onto a scene in another image.

Firstly, the two scenes and the object mask are loaded using *cv2* functions.

**s1** is the scene containing a single object: an ornament elephant.

**s2** is the scene containing two objects: an ornament elephant and a small ornament glass.

**mask** is the object mask for the small ornament glass.

In [2]:
s1 = cv2.imread("Images/1_colour.jpeg", 1) #Scene with 1 object
s2 = cv2.imread("Images/2_colour.jpeg") #Scene with 2 objects
mask = cv2.imread("Images/masks/souvenirs_no_3_colour_mask_2_mask.png") #Target object mask

In [3]:
def ExtractObject(S2, ObjectMask):
    """Extracts an object from an image scene based on the mask used"""

    final_im = ObjectMask*S2
    final_im = cv2.bitwise_not(final_im)
    return final_im

In [4]:
def ApplyFilter(ExtractedObject, FilterIndex):
    """Applies convolution on the object using pre-defined image kernels"""

    if FilterIndex==0: #Apply no filter
        FilteredExObject = ExtractedObject
    elif FilterIndex==1:
        print("")
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        print("")
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        print("")
        #Will need to define some kernels and use them here
    return FilteredExObject

In [5]:
def ObjectBlender(S1, FilteredExObject):
    """Adds the filtered extracted object to the image in scene S1"""

    alpha = 0.5
    beta = 1.0 - alpha
    BlendingResult = cv2.addWeighted(S1, alpha, FilteredExObject, beta, 0.0)
    cv2.imshow('Blended', BlendingResult)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return BlendingResult

In [6]:
def CompareResult(BlendingResult, S2, metric):
    """Compares the blended image with the scene with 2 objects"""

    if metric==1: #Sum of Squared Distance Error (SSD)
        error = BlendingResult-S2
        #error = sum(error(:).^2)
    elif metric==2: #Mean Squared Error (MSE)
        error = np.sum((BlendingResult.astype("float") - S2.astype("float")) ** 2)
        error /= float(BlendingResult.shape[0] * BlendingResult.shape[1])
    return error

In [7]:
extracted = ExtractObject(s2, mask)

In [8]:
blended = ObjectBlender(s1, extracted)

In [9]:
CompareResult(blended, s2, 2)

20031.10287109375

# Stage 2

Stage 2 of this task is concerned with removing the green background from an image and replacing it with another background.

In [15]:
def RemoveGreen(img):
    """Removes the green background from an image"""

    lower_bound = np.array([0, 255, 212]) #Upper bound of green colour
    upper_bound = np.array([0, 255, 128])
    mask = cv2.inRange(img, lower_bound, upper_bound)
    mask = cv2.bitwise_and(mask)

    cv2.imshow("Image without background", mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def NewBackground(imgNoBg, NewBackground):
    """Replaces the background of an image with NewBackground"""

In [16]:
RemoveGreen(s1)

TypeError: bitwise_and() missing required argument 'src2' (pos 2)